<a href="https://colab.research.google.com/github/ssnio/PhaseSlopeIndex.jl/blob/master/tutorials/W11_DeepRL/W11_Tutorial1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIS-522 Week 11 Part 1
# Introduction to Reinforcement Learning

__Instructor:__ Dinesh Jayaraman

__Content creators:__ Chuning Zhu

---

In [ ]:
#@markdown What is your Pennkey and pod? (text, not numbers, e.g. bfranklin)
my_pennkey = '' #@param {type:"string"}
my_pod = 'Select' #@param ['Select', 'euclidean-wombat', 'sublime-newt', 'buoyant-unicorn', 'lackadaisical-manatee','indelible-stingray','superfluous-lyrebird','discreet-reindeer','quizzical-goldfish','astute-jellyfish','ubiquitous-cheetah','nonchalant-crocodile','fashionable-lemur','spiffy-eagle','electric-emu','quotidian-lion']


## Recap the experience from last week

What did you learn last week. What questions do you have? [10 min discussion]

In [ ]:
learning_from_previous_week = '' #@param {type:"string"}

---
# Setup

In [ ]:
# @title Imports

import math
import numpy as np
import IPython
from numbers import Number
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from tqdm.auto import tqdm

In [ ]:
# @title Plotting functions

# Plotting functions
def plot_episode_rewards(episode_rewards):
    fig = plt.figure()
    plt.plot(episode_rewards)
    plt.xlabel("Episode")
    plt.ylabel("Reward")
    fig.show()

In [ ]:
# @title Gridworld Environment

'''
    A custom Gridworld environment with deterministic transitions. Adapted from 
    CS 188 Gridworld env. There are four actions: up, left, down, right. The 
    state is the (x, y) coordinate in the Grid. 
'''
class Gridworld():
    
    def __init__(self, grid, living_reward=-1.0):
        self.h = len(grid)
        self.w = len(grid[0])
        self.living_reward = living_reward
        
        self.scale = math.ceil(max(self.h, self.w) / min(self.h, self.w))
        self.action_space = ['up', 'left', 'down', 'right']
        self.n_actions = 4

        self.init_grid(grid)

    def init_grid(self, grid):
        # Create reward grid. The reward grid is a numpy array storing the
        # reward given for entering each state.
        self.rew_grid = np.array([[self.living_reward if isinstance(e, str) else e 
                                   for e in row] for row in grid], dtype=np.float)

        # Create grid. The grid is a numpy of chars. 
        # S (start), T (terminal), C (cliff), # (block), or ' ' (regular).
        convert_fn = lambda e: 'T' if e >= self.living_reward else 'C'
        self.grid = np.array([[convert_fn(e) if isinstance(e, Number) else e 
                               for e in row] for row in grid])
        
        # Find initial state
        start_indices = np.argwhere(self.grid == 'S')
        if len(start_indices) == 0:
            raise Exception('Grid has no start state')
        self.init_state = (start_indices[0][1], start_indices[0][0])

        
    def get_transition(self, state, action):
        '''
        Execute one action in the environment.
        Args:
            state (tuple): the (x, y) coordinates of the current state.
            action (int): the current action chosen from {0, 1, 2, 3}.

        Returns:
            next_state (tuple): the (x, y) coordinates of the next state.
            reward (float): the reward for the current time step.
        '''
        # Handle terminal states
        x, y = state
        if self.grid[y, x] == 'T':
            return state, 0
        
        # Handle invalid actions
        if action not in range(len(self.action_space)):
            raise Exception('Illegal action')

        # Default transitions
        named_action = self.action_space[action]
        nx, ny = x, y
        if named_action == 'up':
            ny -= 1
        elif named_action == 'left':
            nx -= 1
        elif named_action == 'down':
            ny += 1 
        elif named_action == 'right':
            nx += 1

        # Handle special cases
        if nx < 0 or nx >= self.w or ny < 0 or ny >= self.h or self.grid[ny, nx] == '#':
            # Give living reward if next state is blocked or out of bounds
            reward = self.living_reward
            next_state = (x, y)
        else:
            reward = self.rew_grid[ny, nx]
            if self.grid[ny, nx] == 'C':
                next_state = self.init_state # falls off cliff
            else:
                next_state = (nx, ny) # transition to next state

        return next_state, reward

    def __render(self):
        # Render grid with matplotlib patches. 
        fig, ax = plt.subplots(figsize=(self.h*self.scale, self.w*self.scale))
        ax.set_aspect('equal')
        ax.set_xlim(0, self.w)
        ax.set_ylim(0, self.h)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.tick_params(length=0)
        for y in range(self.h):
            for x in range(self.w):
                cell_type = self.grid[y, x]
                if cell_type == 'S':
                    c = '#DAE8FC' # blue
                elif cell_type == '#':
                    c = '#CCCCCC' # gray
                elif cell_type == 'T':
                    c = '#D5E8D4' # green
                elif cell_type == 'C':
                    c = '#F8CECC' # red
                else:
                    c = '#FFFFFF' # white
                rect = patches.Rectangle((x, self.h-y-1), 1, 1, fc=c, ec='gray', lw=1)
                ax.add_patch(rect)
        return fig, ax

    def render_grid(self):
        fig, ax = self.__render()
        for y in range(self.h):
            for x in range(self.w):
                if self.grid[y, x] != '#':
                    # alternate: x+0.1, self.h-y-0.2
                    ax.text(x+0.5, self.h-y-0.5, str(self.rew_grid[y, x]), size='medium', ha='center', va='center')
        plt.title("Rewards")
        fig.show()
    
    def render_values(self, V):
        fig, ax = self.__render()
        for y in range(self.h):
            for x in range(self.w):
                ax.text(x+0.5, self.h-y-0.5, '{:.2f}'.format(V[y, x]), size='medium', ha='center', va='center')
        plt.title("Values")
        fig.show()

    def render_q_values(self, Q):
        fig, ax = self.__render()
        for y in range(self.h):
            for x in range(self.w):
                named_action = self.action_space[np.argmax(Q[y, x])]
                xl, xc, xr = x, x+0.5, x+1
                yt, yc, yb = self.h-y, self.h-y-0.5, self.h-y-1
                ce, tl, bl, tr, br = [xc, yc], [xl, yt], [xl, yb], [xr, yt], [xr, yb]
                if named_action == 'up':
                    xy = np.array([ce, tl, tr])
                elif named_action == 'left':
                    xy = np.array([ce, tl, bl])
                elif named_action == 'down':
                    xy = np.array([ce, bl, br])
                elif named_action == 'right':
                    xy = np.array([ce, br, tr])
                ax.plot([x, x+1], [self.h-y, self.h-y-1], 'gray', lw=1)
                ax.plot([x, x+1], [self.h-y-1, self.h-y], 'gray', lw=1)
                poly = patches.Polygon(xy, True, fc='#FFFF00', ec='gray')
                ax.add_patch(poly)
                ax.text(x+0.5, self.h-y-0.2, '{:.2f}'.format(Q[y, x, 0]), size='small', ha='center', va='center')
                ax.text(x+0.2, self.h-y-0.5, '{:.2f}'.format(Q[y, x, 1]), size='small', ha='center', va='center')
                ax.text(x+0.5, self.h-y-0.8, '{:.2f}'.format(Q[y, x, 2]), size='small', ha='center', va='center')
                ax.text(x+0.8, self.h-y-0.5, '{:.2f}'.format(Q[y, x, 3]), size='small', ha='center', va='center')
        fig.show()
        plt.title("Q-values")
        pass
    
    def render_policy(self, policy):
        fig, ax = self.__render()
        for y in range(self.h):
            for x in range(self.w):
                if policy[y, x] not in range(len(self.action_space)):
                    raise Exception('Illegal action')
                if self.grid[y, x] == 'T':
                    continue
                arrow_len = 0.3
                dx, dy = 0, 0
                named_action = self.action_space[policy[y, x]]
                if named_action == 'up':
                    dy = arrow_len
                elif named_action == 'left':
                    dx = -arrow_len
                elif named_action == 'down':
                    dy = -arrow_len
                elif named_action == 'right':
                    dx = arrow_len
                arrow = patches.FancyArrow(x+0.5, self.h-y-0.5, dx, dy, 0.03, True, color='#6C8EBF')
                ax.add_patch(arrow)
        plt.title("Policy")
        fig.show()


class GridworldEnv(Gridworld):
    def __init__(self, grid, living_reward=-1.0):
        super().__init__(grid, living_reward)
        self.reset()
        
    def reset(self):
        '''
        Set the agent to the initial state
        '''
        self.state = self.init_state

    def step(self, action):
        '''
        Execute one action in the environment.
        Args:
            action (int): the current action chosen from {0, 1, 2, 3}.

        Returns:
            next_state (tuple): (x, y) coordinates of the next state.
            reward (float): reward for the current time step.
            done (bool): True if a terminal state has been reached, False otherwise.
        '''
        next_state, reward = self.get_transition(self.state, action)
        self.state = next_state
        done = self.grid[self.state[1], self.state[0]] == 'T'
        return next_state, reward, done

# Pre-defined grids
def get_book_grid():
    grid = [['T',  ' ',  ' ',  ' '],
            [' ',  ' ',  ' ',  ' '],
            [' ',  ' ',  ' ',  ' '], 
            ['S',  ' ',  ' ',  'T']]
    return GridworldEnv(grid)

def get_cliff_small():
    grid = [[' ',   ' ',  ' ',  ' ',  ' '],
            ['S',   ' ',  ' ',  ' ',   'T'],
            [-100, -100, -100, -100, -100]]
    return GridworldEnv(grid)

def get_cliff_walk():
    grid = [[' ' for _ in range(12)] for _ in range(3)]
    grid.append([-100 for _ in range(12)])
    grid[3][ 0] = 'S'
    grid[3][-1] = 'T'
    return GridworldEnv(grid)

def get_bridge_grid():
    grid = [[ '#',-100, -100, -100, -100, -100, '#'],
            [   1, 'S',  ' ',  ' ',  ' ',  ' ',  10],
            [ '#',-100, -100, -100, -100, -100, '#']]
    return GridworldEnv(grid)

---
# Section 1: Introduction


In [ ]:
#@title Video : Intro to Reinforcement Learning
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="cVTud58UfpQ", width=854, height=480, fs=1)
print("Video available at https://youtube.com/watch?v=" + video.id)

video

Up to this point, we have mainly been concerned with supervised learning. In a supervised learning problem, we are provided with a dataset where each sample comes with a ground truth label (e.g. class label), and the goal is to learn to predict the label by minimizing some loss function. Reinforcement learning, on the other hand, is a framework for solving sequential decision-making problems. Consider an agent operating in some environment. The agent's goal is to carry out the best sequence of actions that maximizes the cumulative reward. This is difficult because the action at the current time step influences future states of the environment, which then feed back to the agent's observations. The following figure illustrates this setting. 

![RL setting](https://drive.google.com/uc?export=view&id=1vpqNaoXcrgT1jBNEDtLSHNNVtfaAhfIk)


What is the role of reinforcement learning in intelligence? According to Yann LeCun, if intelligence is a cake, then unsupervised learning is the bulk of the cake, supervised learning the icing, and reinforcement learning the cherry on top. The reason RL takes up such a small proportion is that very little learning in real world comes with explicit reward signal. This analogy is still debatable, as some RL folks argue that intelligence is more like a cake with lots of cherries on top, especially after the invention of [hindight experience replay](https://arxiv.org/abs/1707.01495). In addition, there are ways to solve sequential decision making problems without relying on shaped rewards, such as inverse reinforcement learning, which infers a reward function from experience, and learning from goals / demonstrations / examples. Another way to put RL in perspective is by comparing it with vision and natural language processing. If we decompose intelligence into perception, cognition (reasoning), and action (decision making), then vision coarsely corresponds to perception, NLP cognition, and RL action. Just like how vision can be combined with NLP for tasks like image captioning, RL can be organically combined with vision and NLP as well. 

![Cherry cake](https://drive.google.com/uc?export=view&id=1h-2tu_mItVfObpBus3EiNSbhSxto8hru)

In this first tutorial, we will briefly step away from deep learning and study a few classic approaches in reinforcement learning. A good reference is Sutton and Barto's book, Reinforcement Learning: An Introduction. The [full text](http://incompleteideas.net/book/the-book.html) is avaliable online.

---
# Section 2: MDP and Bellman Equations

In [ ]:
#@title Video : Markov Decision Processes
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="GJEL-QkT2yk", width=854, height=480, fs=1)
print("Video available at https://youtube.com/watch?v=" + video.id)

video

In [ ]:
#@title Video : Solving MDPs
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="meywaLPitZ4", width=854, height=480, fs=1)
print("Video available at https://youtube.com/watch?v=" + video.id)

video


## Section 2.1: Markov Decision Process

We begin our study of reinforcement learning with a definition of  Markov decision process. A Markov decision process (MDP) is a tuple $(S, A, P, R, \gamma)$, where

- $S$ is the set of **states**.
- $A$ is the set of **actions**.
- $P$ defines the **transition probablities**. $P(s'|s, a)$ gives the probability of transitioning to state $s'$ by taking action $a$ at state $s$. 
- $R$ is the **reward function**. $R(s, a)$ gives the reward of taking action $a$ at state $s$. $R$ can also be a function of state only.
- $\gamma$ is the **discount factor**. It controls how much future rewards matter to us.

A policy $\pi$ is a function mapping states to distributions over actions. At state $s$, we sample an action from the distribution $\pi(a|s)$ to execute in the environment. If all probability mass is assigned to a single action, then the policy is deterministic. The goal of reinforcement learning is to find a policy that maximizes the expected sum of discounted rewards: $$E\left[\sum_{t=0}^{\infty}\gamma^tR(s_t, a_t)\right]$$

Note that this objective assumes a continuous task, i.e. that $t$ extends to infinity. We can easily generalize it to episodic tasks with finite horizons by replacing $\infty$ with task horizon $T$.


As an aside, we introduce partially observable MDP (POMDP). A POMDP additionally has a set of obervations $O$ and emission probabilities $\varepsilon$. $\varepsilon(o|s)$ gives the probability of observing $o$ at state $s$. This formulation is useful when we don't have access to explicit state information, but are provided with observations that may not fully reveal the underlying states. An example is reinforcement learning from images.


Come up with a sequential decision making problem and formalize it as an MDP. What are $S$, $A$, $P$, and $R$? Share your example with your pod.

In [ ]:
MDP_example = '' #@param {type:"string"}

## Section 2.2: Bellman Equations




In [ ]:
#@title Video : V, Q, and the Bellman Equation
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="tm39P5jT320", width=854, height=480, fs=1)
print("Video available at https://youtube.com/watch?v=" + video.id)

video

We define the value of a state $s$ under policy $\pi$ as the expected future reward for following $\pi$ starting from $s$: $$V^{\pi}(s) = E_{\pi} \left[\sum_{t'=t}^{\infty} \gamma^{t'-t}R(s_{t'}, a_{t'}) \mid s_t = s\right] $$

We further define the value of a state-action pair $(s, a)$ under policy $\pi$ as the expected future reward for taking action $a$ at state $s$, *and then* following $\pi$. This is also known as the Q-value. $$Q^{\pi}(s, a) = E_{\pi} \left[\sum_{t'=t}^{\infty} \gamma^{t'-t}R(s_{t'}, a_{t'}) \mid s_t = s, a_t = a\right]$$

Observe that $V$ and $Q$ can be related by a simple equation:
$$V^{\pi}(s) = E_{a\sim \pi(a|s)}\left[Q^{\pi}(s, a)\right]$$

By definition, $V$ and $Q$ satisfy the following Bellman equations.

\begin{align*}
V^{\pi}(s) &= E_{a \sim \pi(a|s)}\left[R(s, a)+ \gamma E_{s' \sim P(s'|s, a)} \left[V^{\pi}(s')\right]\right] \\
Q^{\pi}(s, a) &= R(s, a) + \gamma E_{s' \sim P(s'|s, a)}\left[ E_{a' \sim \pi(a'|s')}\left[Q(s', a')\right]\right]
\end{align*}

The optimal value function capture the expected future reward if we start from state $s$ and act optimally in the future. Similarly, the optimal Q-function captures the expected future reward if we start from state $s$, take action $a$, and then act optimally in the future. They satisfy the Bellman optimality equations:   

\begin{align*}
V^*(s) &= \max_{a\in A}\left(R(s, a) + \gamma E_{s' \sim P(s'|s, a)} \left[V^*(s')\right]\right)\\
Q^*(s, a) &= R(s, a) + \gamma E_{s' \sim P(s'|s, a)} \left[ \max_{a' \in A} Q^*(s', a')\right]
\end{align*}

If we have learned the optimal value function $V^*$ or Q-function $Q^*$, we can infer an optimal (deterministic) policy known as the greedy policy or argmax policy: $$\pi(s) = \arg\max_{a\in A}Q^*(s, a)$$

---
# Section 3: Gridworld


Before we dive into RL algorithms, let's get familiar with the running example we will use throughout this tutorial -- the Gridworld environment. As its name suggests, the Gridworld environment is an $m \times n$ grid. The states are xy coordinates in the grid, with origin at the top-left corner. The agent starts from the initial state and aims to reach the terminal state. There are four actions: up (0), left (1), down (2), and right (3). Each action leads to a **deterministic** transition to the adjacent cell in the correspond direction. By default, a reward of -1 is issued for entering any non-terminal state, although our implementation allows you to define an arbitrary reward for each state. 

To be more specific, our implementation admits four special cells: 'S' (start), 'T' (terminal), 'C' (cliff), and '#' (block). They are colored blue, green, red, and gray respectively. The agent spawns at the start cell, and aims to reach the terminal cell. If the agent "falls off the cliff," it will get a high penalty (-100) and be sent back to the start cell. If the agent tries to enter a block cell or go out of the grid, it will instead stay at the same place and get a reward of -1. 

Familiarize yourself with the environment by reading the code for Gridworld and GridworldEnv (feel free to ignore the rendering functions) and interacting with the following code cells. 

In [ ]:
# Get a pre-defined grid
gw = get_cliff_small()

# Render rewards 
gw.render_grid()

# Render random values
values = np.random.rand(4, 12)
gw.render_values(values)

# Render random Q-values and argmax policy
q_values = np.random.randn(4, 12, 4)
gw.render_q_values(q_values)

# Render random policy
policy = np.random.choice(4, (4, 12)).astype(int)
gw.render_policy(policy)

In [ ]:
# Interact with the environment to see if you can get to the terminal state.
# We don't have a visualization for the agent on the grid, so you may have 
# to use a bit of imagination.
print(gw.step(0))
print(gw.step(2))

---
# Section 4 Dynamic Programming

In [ ]:
#@title Video : Policy and Value Iteration
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="l87rgLg90HI", width=854, height=480, fs=1)
print("Video available at https://youtube.com/watch?v=" + video.id)

video

## Section 4.1: Policy Iteration

If we have full knowledge of the environment, in particular its transitions, we can use dynamic programming to find the optimal policy. The first algorithm we will study is policy iteration. We start with policy evaluation, which computes the value function of the policy using the Bellman equation. We iteratively perform Bellman backup for the value of each state until convergence: 
$$V(s) \leftarrow \sum_{a} \pi(a|s) \left(R(s, a) + \gamma\sum_{s'}P(s'|s, a)V(s')\right) $$
Since we have deterministic transitions, this simplifies to 
$$V(s) \leftarrow \sum_{a} \pi(a|s) \left(R(s, a) + \gamma V(s')\right)$$
where $s'$ is the state we transition to by taking action $a$ at state $s$. 

In the following excercise, you will evaluate a random policy which assigns equal probablities to all actions at each state. Complete one step of Bellman backup. You can get the next state and reward using grid.get_transition((x, y), action).


### Exercise 1

In [ ]:
# Random Policy evaluation
def random_policy_evaluation(grid, gamma=1.0):
    values = np.zeros_like(grid.rew_grid)
    iter = 0
    while True:
        eps = 0
        for y in range(grid.h):
            for x in range(grid.w):
                v = values[y, x]
                new_v = 0
                for action in range(grid.n_actions):
                     ####################################################################
                     # Fill in missing code below (...),
                     # then remove or comment the line below to test your function
                     raise NotImplementedError("Random policy evaluation")
                     ####################################################################
                     new_v += ...
                     
                values[y, x] = new_v
                eps = max(eps, abs(new_v - v))
        iter += 1
        if eps < 0.0001:
            print("Converged after {} iterations".format(iter))
            break
    return values

# # Uncomment to test
# grid = get_book_grid()
# values = random_policy_evaluation(grid)
# grid.render_values(values)

In [ ]:
# to_remove solution

# Random Policy evaluation
def random_policy_evaluation(grid, gamma=1.0):
    values = np.zeros_like(grid.rew_grid)
    iter = 0
    while True:
        eps = 0
        for y in range(grid.h):
            for x in range(grid.w):
                v = values[y, x]
                new_v = 0
                for action in range(grid.n_actions):
                    (new_x, new_y), reward = grid.get_transition((x, y), action)
                    new_v += 0.25 * (reward + gamma * values[new_y, new_x])
                values[y, x] = new_v
                eps = max(eps, abs(new_v - v))
        iter += 1
        if eps < 0.0001:
            print("Converged after {} iterations".format(iter))
            break
    return values

grid = get_book_grid()
values = random_policy_evaluation(grid)
grid.render_values(values)

Now we move on to the policy iteration algorithm. Policy iteration consists of two steps: policy evaluation and policy improvement. We first evaluate the policy, and then use the new values to derive a better policy by selecting the greedy action at each state. These steps are repeated until convergence. For an analysis of the theoretical guarantees of policy iteration, see [this page](http://incompleteideas.net/book/first/ebook/node42.html).

In the following exercise, you will implement the policy iteration algorithm. For policy evaluation, note that we have a deterministic greedy policy, so there's no need to iterate over actions. The backup thus becomes $V(s) \leftarrow R(s, \pi(s)) + \gamma V(s')$. For policy improvement, we do the same evaluation for all actions and store them in the action_values array, from which we derive the greedy policy. **Be careful when indexing into the value matrix**: values[y, x] stores the value of state (x, y).

### Exercise 2

In [ ]:
# Policy Iteration 
def policy_evaluation(grid, values, policy, gamma):
    while True:
        eps = 0
        for y in range(grid.h):
            for x in range(grid.w):
                v = values[y, x]
                ####################################################################
                # Fill in missing code below (...),
                # then remove or comment the line below to test your function
                raise NotImplementedError("Policy evaluation")
                ####################################################################
                new_v = ...
                
                values[y, x] = new_v
                eps = max(eps, abs(new_v - v))
        if eps < 0.0001: 
            break

def policy_improvement(grid, values, policy, gamma):
    converged = True
    for y in range(grid.h):
        for x in range(grid.w):
            old_action = policy[y, x]
            action_values = np.zeros(grid.n_actions, dtype=np.float)
            ####################################################################
            # Fill in missing code below (...),
            # then remove or comment the line below to test your function
            raise NotImplementedError("Policy improvement")
            ####################################################################
            for action in range(...):
                action_values[action] = ...
            policy[y, x] = ...

            if old_action != policy[y, x]:
                converged = False
    return converged

def policy_iteration(grid, gamma=0.9):
    policy = np.random.choice(grid.n_actions, (grid.h, grid.w)).astype(int)
    values = np.zeros_like(grid.rew_grid)
    converged = False
    while not converged:
        print("running policy evaluation")
        policy_evaluation(grid, values, policy, gamma)
        print("running policy improvement")
        converged = policy_improvement(grid, values, policy, gamma)
    return values, policy

# # Uncomment to test
# grid = get_book_grid()
# values, policy = policy_iteration(grid)
# grid.render_values(values)
# grid.render_policy(policy)

In [ ]:
# to_remove solution

# Policy Iteration 
def policy_evaluation(grid, values, policy, gamma):
    while True:
        eps = 0
        for y in range(grid.h):
            for x in range(grid.w):
                v = values[y, x]
                (new_x, new_y), reward = grid.get_transition((x, y), policy[y, x])
                new_v = reward + gamma * values[new_y, new_x]
                values[y, x] = new_v
                eps = max(eps, abs(new_v - v))
        if eps < 0.0001: 
            break

def policy_improvement(grid, values, policy, gamma):
    converged = True
    for y in range(grid.h):
        for x in range(grid.w):
            old_action = policy[y, x]
            action_values = np.zeros(grid.n_actions, dtype=np.float)
            for action in range(grid.n_actions):
                (new_x, new_y), reward = grid.get_transition((x, y), action)
                action_values[action] = reward + gamma * values[new_y, new_x]
            policy[y, x] = np.argmax(action_values)
            if old_action != policy[y, x]:
                converged = False
    return converged

def policy_iteration(grid, gamma=0.9):
    policy = np.random.choice(grid.n_actions, (grid.h, grid.w)).astype(int)
    values = np.zeros_like(grid.rew_grid)
    converged = False
    while not converged:
        print("running policy evaluation")
        policy_evaluation(grid, values, policy, gamma)
        print("running policy improvement")
        converged = policy_improvement(grid, values, policy, gamma)
    return values, policy


grid = get_book_grid()
values, policy = policy_iteration(grid)
grid.render_values(values)
grid.render_policy(policy)

Construct the path from the policy visualization and see that following the policy from the initial state indeed leads to terminal state. Now change $\gamma$ to 1.0 and rerun the code. Does policy iteration still converge? Why are we stuck on policy evaluation? (This is a brain-teaser, so don't spend too much time on it, and don't let the code run for too long.)

In [ ]:
convergence = '' #@param {type:"string"}

In [ ]:
# to_remove solution
# Policy iteration fails to converge because we're stuck at the first policy 
# evaluation step. The initial policy is all zeros, which means we take action 
# "up" at all states. Consider what happens at the top left corner. Going up at 
# this state will lead to a reward of -1 and transitioning back to itself. Since 
# gamma = 1, we have V(s) = V(s) - 1, which never converges. This can be avoided 
# by setting gamma < 1. The evaluation will halt when V(s) = gamma * V(s) - 1.

## Section 4.2: Value Iteration

Value iteration can be thought of as a simplification of policy iteration, where we effectively combine the two steps in policy iteration into one. We still iterate over all states, but in each iteration the value update becomes $$V(s) \leftarrow \max_a R(s, a) + \gamma\sum_{s'}P(s'|s, a)V(s') $$ So instead of computing the state value and then selecting the greedy action, we directly store the maximum state-action value. This obviates the need to maintain an explicit policy. After the value matrix has converged, we can back out the optimal policy by taking the argmax, same as what we did in policy improvement.

Now it's your turn to implement the value iteration algorithm. You need to fill in the new update rule, and copy your code from policy improvment to reconstruct the optimal policy.

### Exercise 3

In [ ]:
# Value Iteration

def value_iteration(grid, gamma=0.9):
    V = np.zeros_like(grid.rew_grid)
    while True:
        eps = 0
        for y in range(grid.h):
            for x in range(grid.w):
                v = values[y, x]
                action_values = np.zeros(grid.n_actions)
                ####################################################################
                # Fill in missing code below (...),
                # then remove or comment the line below to test your function
                raise NotImplementedError("Value iteration")
                ####################################################################
                for action in range(...):
                    action_values[action] = ... 
                new_v = ...
                
                values[y, x] = new_v
                eps = max(eps, abs(new_v - v))
        if eps < 0.0001:
            break

    # Create greedy policy from values
    policy = np.zeros_like(grid.rew_grid).astype(int)
    for y in range(grid.h):
        for x in range(grid.w):
            action_values = np.zeros(grid.n_actions)
            ####################################################################
            # Copy your solution for policy improvement here
            raise NotImplementedError("Value iteration policy")
            #################################################################### 
            for action in range(...):
                action_values[action] = ...
            policy[y, x] = ...

    return values, policy

# # Uncomment to test
# grid = get_book_grid()
# values, policy = value_iteration(grid)
# grid.render_values(values)
# grid.render_policy(policy)

In [ ]:
# to_remove solution

# Value Iteration
def value_iteration(grid, gamma=0.9):
    V = np.zeros_like(grid.rew_grid)
    while True:
        eps = 0
        for y in range(grid.h):
            for x in range(grid.w):
                v = values[y, x]
                action_values = np.zeros(grid.n_actions)
                for action in range(grid.n_actions):
                    (nx, ny), reward = grid.get_transition((x, y), action)
                    action_values[action] = reward + gamma * values[ny, nx] 
                new_v = np.max(action_values)
                values[y, x] = new_v
                eps = max(eps, abs(new_v - v))
        if eps < 0.0001:
            break

    # Create greedy policy from values
    policy = np.zeros_like(grid.rew_grid).astype(int)
    for y in range(grid.h):
        for x in range(grid.w):
            action_values = np.zeros(grid.n_actions)
            for action in range(grid.n_actions):
                (nx, ny), reward = grid.get_transition((x, y), action)
                action_values[action] = reward + gamma * values[ny, nx] 
            policy[y, x] = np.argmax(action_values)

    return values, policy

grid = get_book_grid()
values, policy = value_iteration(grid)
grid.render_values(values)
grid.render_policy(policy)

---
# Section 5: Temporal Difference (TD) Learning



In [ ]:
#@title Video : TD and Q Learning
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="rCk_hvwZ6iA", width=854, height=480, fs=1)
print("Video available at https://youtube.com/watch?v=" + video.id)

video

## Section 5.1 Q-learning

Up until now we have assumed full access to the transitions of an environment. But in a typical reinforcement learning problem the dynamics is unknown. So how do we solve it? One way is to learn to approximate the dynamics using a function approximator (e.g. a neural net) and then apply dynamic programming or trajectory optimization. This is called model-based reinforcement learning, which we will cover next week. In this tutorial, we will study algorithms in the model-free regime. Specifically, we will investigate **Temporal Difference (TD) learning**.

The idea behind TD learning is to use $V(s_{t+1})$ as an imperfect proxy for the true value (Monte Carlo bootstrapping), and obtain a generalized equation to calculate the TD error:
$$\delta_t = r_{t+1} + \gamma V(s_{t+1}) - V(s_t)$$

The expression $r_{t+1} + \gamma V(s_{t+1})$ is also called the TD target. We can then update the value using a learning rate $\alpha$.
$$ V(s_t) \leftarrow V(s_t) + \alpha \delta_t$$

**Q-learning** is an instantiation of TD learning, where the TD error is $$\delta_t = R(s_t, a_t) + \gamma \max_{a} Q(s_{t+1}, a_{t+1}) - Q(s_t, a_t)$$ and the full update rule is $$Q(s_t,a_t) \leftarrow Q(s_t, a_t) + \alpha \left(R(s_t, a_t) + \gamma \max_{a} Q(s_{t+1}, a_{t+1}) - Q(s_t, a_t)\right)$$

Because of the max operator used to select the optimal Q-value in the TD target, Q-learning directly estimates the optimal action value, i.e. the cumulative future reward that would be obtained if the agent behaved optimally, regardless of the policy currently followed by the agent. For this reason, Q-learning is referred to as an **off-policy** method. A sketch of the Q-learning algorithm is as follows:

```
for n episodes:
    for T steps:
        Select an action a_t using some policy derived from the current Q-values
        Execute a_t in the environment to get reward r and next state s_{t+1}
        Update Q(s_t, a_t) using (s_t, a_t, r, s_{t+1})
```

A remaining question is, how do we select an action base on the current Q-values? If the approximated Q-values are very bad, then greedily following the argmax policy may cause the agent to get stuck in some bad states. Thus, we instead adopt an **epsilon-greedy policy**, where we choose the argmax action with probability $(1-\epsilon)$ and take a random action otherwise. This relates to an important concept in reinforcement learning, namely exploration vs. exploitation. 

In [ ]:
# Epsilon-greedy policy
def epsilon_greedy(q_values, epsilon):
    if np.random.random() > epsilon:
        action = np.argmax(q_values)
    else:
        action = np.random.choice(len(q_values))
    return action

# General TD learning algorithm
def learn_gridworld(env, backup_rule, params, max_steps, n_episodes):
    values = np.zeros((env.h, env.w, env.n_actions))
    episode_actions = []
    episode_rewards = np.zeros(n_episodes)

    for episode in tqdm(range(n_episodes)):
        env.reset()
        total_reward = 0
        action_list = []

        for t in range(max_steps):
            state = env.state
            # Select action from epsilon-greedy policy
            action = epsilon_greedy(values[state[1], state[0]], params['epsilon'])
            action_list.append(action)
            # Execute action
            next_state, reward, done = env.step(action)
            # Update values
            values = backup_rule(state, action, reward, next_state, values, params)
            total_reward += reward
            if done:
                break
        
        episode_actions.append(action_list)
        episode_rewards[episode] = total_reward

    return values, episode_rewards

### Exercise 4

In this exercise, you will implement the update rule for Q-learning and test it on the Cliff World environment, where the agent needs to navigate to the other side of the cliff without falling off. You need to fill in the code for computing the TD error and updating the values matrix.

In [ ]:
# Q-Learning
def q_learning_backup(state, action, reward, next_state, values, params):
    '''
    Compute a new set of q-values using the q-learning update rule.
    Args:
        state (tuple): s_t, a tuple of xy coordinates.
        action (int): a_t, an integer from {0, 1, 2, 3}.
        reward (float): the reward of executing a_t at s_t.
        next_state (tuple): s_{t+1}, a tuple of xy coordinates.
        values (ndarray): an (h, w, 4) numpy array of q-values. values[y, x, a] 
                          stores the value of executing action a at state (x, y).
        params (dict): a dictionary of parameters.

    Returns:
        ndarray: the updated q-values.
    '''
    x, y = state
    nx, ny = next_state
    gamma = params['gamma']
    alpha = params['alpha']

    ####################################################################
    # Fill in missing code below (...),
    # then remove or comment the line below to test your function
    raise NotImplementedError("Q-learning")
    ####################################################################
    q = ...
    max_next_q = ...

    # Compute TD error using q and max_next_q
    td_error = ...
    values[y, x, action] = ...

    return values

# # Uncomment to test
# env = get_cliff_walk()
# params = {'gamma': 1.0, 'alpha': 0.1 , 'epsilon': 0.1}
# max_steps = 1000
# n_episodes = 500
# q_values, episode_rewards = learn_gridworld(env, q_learning_backup, params, max_steps, n_episodes)

# plot_episode_rewards(episode_rewards)
# env.render_policy(np.argmax(q_values, axis=2))
# env.render_q_values(q_values)

In [ ]:
# to_remove solution

# Q-learning
def q_learning_backup(state, action, reward, next_state, values, params):
    x, y = state
    nx, ny = next_state
    gamma = params['gamma']
    alpha = params['alpha']
    
    q = values[y, x, action]
    max_next_q = np.max(values[ny, nx])

    td_error = reward + (gamma * max_next_q - q)
    values[y, x, action] = q + alpha * td_error

    return values

env = get_cliff_walk()
params = {'gamma': 1.0, 'alpha': 0.1 , 'epsilon': 0.1}
max_steps = 1000
n_episodes = 500
q_values, episode_rewards = learn_gridworld(env, q_learning_backup, params, max_steps, n_episodes)

plot_episode_rewards(episode_rewards)
env.render_policy(np.argmax(q_values, axis=2))
env.render_q_values(q_values)

## Section 5.2: SARSA

An alternative to Q-learning, the SARSA algorithm also estimates action values. However, rather than estimating the optimal (off-policy) values, SARSA estimates the **on-policy** action values, i.e. the cumulative future reward that would be obtained if the agent behaved according to its current beliefs.

\begin{align}
Q(s_t,a_t) \leftarrow Q(s_t,a_t) + \alpha \big(R(s_t, a_t) + \gamma Q(s_{t+1}, \pi(s_{t+1})) - Q(s_t,a_t)\big)
\end{align}

In fact, you will notices that the *only* difference between Q-learning and SARSA is the TD target calculation uses the policy to select the next action (in our case epsilon-greedy) rather than using the action that maximizes the Q-value. You do not need to implement the SARSA algorithm. Run the following code cell and compare with Q-learning.

In [ ]:
# SARSA
def sarsa_backup(state, action, reward, next_state, values, params):
    '''
    Compute a new set of q-values using the SARSA update rule.
    Args:
        state (tuple): s_t, a tuple of xy coordinates.
        action (int): a_t, an integer from {0, 1, 2, 3}.
        reward (float): the reward of executing a_t at s_t.
        next_state (tuple): s_{t+1}, a tuple of xy coordinates.
        values (ndarray): an (h, w, 4) numpy array of q-values. values[y, x, a] 
                          stores the value of executing action a at state (x, y).
        params (dict): a dictionary of parameters.

    Returns:
        ndarray: the updated q-values.
    '''
    x, y = state
    nx, ny = next_state
    gamma = params['gamma']
    alpha = params['alpha']

    q = values[y, x, action]
    # Obtain on-policy action 
    policy_action = epsilon_greedy(values[ny, nx], params['epsilon'])
    next_q = values[ny, nx, policy_action]

    # Compute TD error using q and max_next_q
    td_error = reward + (gamma * next_q - q)
    values[y, x, action] = q + alpha * td_error

    return values

env = get_cliff_walk()
params = {'gamma': 1.0, 'alpha': 0.1 , 'epsilon': 0.1}
max_steps = 1000
n_episodes = 500
q_values, episode_rewards = learn_gridworld(env, sarsa_backup, params, max_steps, n_episodes)

plot_episode_rewards(episode_rewards)
env.render_policy(np.argmax(q_values, axis=2))
env.render_q_values(q_values)

Compare the reward plots and policies of Q-learning and SARSA. Do they take the same path to reach the terminal state? Why does one look more conservative than the other?

In [ ]:
q_vs_sarsa = '' #@param {type:"string"}

In [ ]:
# to_remove solution
# SARSA appears to be more conservative than Q-learning. The path taken by SARSA
# is one step away from the cliff, whereas that taken by Q-learning is right on
# the edge. We also see this behavior from the higher variance in the reward plot 
# of Q-learning. The reason is that SARSA constructs its TD target by taking one 
# more on-policy action. Consider updating the value of action "right" at an edge 
# cell. The TD target for Q-learning involves the value of the next greedy action, 
# which will converge to "right" after some iterations. The TD target for SARSA 
# involves the value of the next epsilon-greedy action, which always has a chance 
# of being "down" aka "falling off the cliff." Thus, SARSA assign a lower values 
# to action "right" for edge cells.

## Section 5.3 (Optional): Try your own grid

If time allows, you are welcome to try Q-learning or SARSA on one of the other pre-defined grids or a Gridworld of your own creation. Discuss your findings with your pod.

# Section 5: Wrap-up and foreshadowing

In [ ]:
#@title Video : Wrap-up
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="oJo0jb_h2sM", width=854, height=480, fs=1)
print("Video available at https://youtube.com/watch?v=" + video.id)

video